In [1]:
import pandas as pd
import numpy as np
data_test_awal = pd.read_csv('test.csv')
data_train_awal = pd.read_csv('train.csv')


In [2]:
data_test = data_test_awal.replace('?', np.nan)
data_train = data_train_awal.replace('?', np.nan)

In [3]:
display(data_train.iloc[8])

id                                          8
Umur                                       23
Kelas Pekerja                             NaN
Berat Akhir                            191910
Pendidikan                  Pendidikan Tinggi
Jmlh Tahun Pendidikan                      10
Status Perkawinan        Belum Pernah Menikah
Pekerjaan                                 NaN
Jenis Kelamin                           Laki2
Keuntungan Kapital                          0
Kerugian Capital                            0
Jam per Minggu                             40
Gaji                                    <=7jt
Name: 8, dtype: object

In [4]:
data_test['Pekerjaan'].value_counts()

Perbaikan Kerajinan      1205
Ekesekutif Managerial    1188
Spesialis                1142
Pemuka Agama             1112
Sales                    1079
Servis Lainnya            982
Mesin Inspeksi            590
Supir                     458
Pembersih                 398
Petani                    316
Tech-support              301
Penjaga                   228
Asisten Rumah Tangga       43
Tentara                     3
Name: Pekerjaan, dtype: int64

In [5]:
data_test['Kelas Pekerja'].value_counts()

Wiraswasta                       6714
Pekerja Bebas Bukan Perusahan     732
Pemerintah Lokal                  624
Pemerintah Negara                 376
Pekerja Bebas Perusahaan          316
Pemerintah Provinsi               279
Tanpa di Bayar                      4
Tidak Pernah Bekerja                2
Name: Kelas Pekerja, dtype: int64

In [6]:
data_train['Pekerjaan'] = data_train['Pekerjaan'].fillna('Spesialis')
data_train['Kelas Pekerja'] = data_train['Kelas Pekerja'].fillna('Wiraswasta')

data_test['Pekerjaan'] = data_test['Pekerjaan'].fillna('Perbaikan Kerajinan')
data_test['Kelas Pekerja'] = data_test['Kelas Pekerja'].fillna('Wiraswasta')

In [7]:
x_train = data_train.drop('Gaji', axis=1)
y_train = data_train['Gaji']

In [8]:
obj_dict = {
    '<=7jt':0,
    '>7jt':1
}
y_train_convert = y_train.replace(obj_dict)
x_train_convert = pd.get_dummies(x_train, columns=['Kelas Pekerja', 'Status Perkawinan', 'Pekerjaan', 'Jenis Kelamin'])

In [9]:
obj_dict2 = {
    'SD':0,
    '1st-4th':1,
    '5th-6th':2,
    '7th-8th':3,
    '9th':4,
    '10th':5,
    '11th':6,
    '12th':7,
    'SMA':8,
    'Pendidikan Tinggi':9,
    'D4':10,
    'D3':11,
    'Sarjana':12,
    'Master':13,
    'Sekolah Professional':14,
    'Doktor':15, 
}

In [10]:
x_train_convert['Pendidikan'] = x_train['Pendidikan'].replace(obj_dict2)


In [11]:
data_test_convert = pd.get_dummies(data_test, columns=['Kelas Pekerja', 'Status Perkawinan', 'Pekerjaan', 'Jenis Kelamin'])
data_test_convert['Pendidikan'] = data_test['Pendidikan'].replace(obj_dict2)

In [12]:
# data standardization with  sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = x_train_convert.copy()
X_test_stand = data_test_convert.copy()

# numerical features
num_cols = ['id','Umur','Berat Akhir','Pendidikan','Jmlh Tahun Pendidikan','Keuntungan Kapital','Kerugian Capital','Jam per Minggu']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

In [13]:
# training a KNN model
from sklearn.neighbors import KNeighborsRegressor
# measuring RMSE score
from sklearn.metrics import mean_squared_error

# knn 
knn = KNeighborsRegressor(n_neighbors=57, weights='uniform')


# raw, normalized and standardized training and testing data
#trainX = [X_train, X_train_norm, X_train_stand]
#testX = [X_test, X_test_norm, X_test_stand]

# model fitting and measuring RMSE
#for i in range(len(trainX)):
    
    # fit
knn.fit(X_train_stand, y_train_convert)
    # predict
pred = knn.predict(X_test_stand)

In [56]:
pred

array([0.        , 0.14814815, 0.55555556, ..., 0.        , 0.        ,
       0.11111111])

In [57]:
import numpy as np
pred2 = np.round(pred, 0)

In [58]:
pred2

array([0., 0., 1., ..., 0., 0., 0.])

In [59]:
pred3 = pred2.astype(int)

In [60]:
data_test_id = data_test['id']
hasil_prediksi = pd.DataFrame(pred3, columns=['Gaji']) 

list_dataframe = [data_test_id, hasil_prediksi]
# gabungkan data dengan metode concat dari pandas
hasil_akhir = pd.concat(list_dataframe, axis=1, ignore_index=True)

hasil_akhir.columns = ['id','Gaji']

In [61]:
hasil_akhir.to_csv('percobaan9.csv', index=False)

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

#x_reshape = x.reshape(-1, 1)
#x_new = x_new.reshape(-1, 1)

model = DecisionTreeRegressor(max_depth=5, min_samples_leaf=0.1, random_state=3)
model.fit(X_train_stand, y_train_convert)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=0.1, random_state=3)

In [14]:
y_test = model.predict(X_test_stand)

In [20]:
y_test = np.round(y_test, 0)
y_test = y_test.astype(int)

In [21]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [22]:
data_test_id = data_test['id']
hasil_prediksi = pd.DataFrame(y_test, columns=['Gaji']) 

list_dataframe = [data_test_id, hasil_prediksi]
# gabungkan data dengan metode concat dari pandas
hasil_akhir = pd.concat(list_dataframe, axis=1, ignore_index=True)

hasil_akhir.columns = ['id','Gaji']

In [23]:
hasil_akhir.to_csv('percobaan6.csv', index=False)